# Part 1: Features Engineering

This section explains what features has been engineered and why:
- ma7
- ma21
- rsi
- macd
- bollinger bands
- volatility
- Change
- Return (daily)
- news headlines (tesla)

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from importlib import reload
from features_engineering import ma7, ma21, rsi, macd, bollinger_bands, momentum, get_tesla_headlines

from bs4 import BeautifulSoup
import requests
from nltk.sentiment.vader import SentimentIntensityAnalyzer
warnings.filterwarnings('ignore')
plt.rcParams['figure.dpi'] = 227 # native screen dpi for my computer

# Original Data

In [2]:
tesla_df = pd.read_csv('data/raw_stocks/tsla.csv').head()

In [3]:
tesla_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-06-29,19.000000,25.00,17.540001,23.889999,23.889999,18766300
1,2010-06-30,25.790001,30.42,23.299999,23.830000,23.830000,17187100
2,2010-07-01,25.000000,25.92,20.270000,21.959999,21.959999,8218800
3,2010-07-02,23.000000,23.10,18.709999,19.200001,19.200001,5139800
4,2010-07-06,20.000000,20.00,15.830000,16.110001,16.110001,6866900


In [4]:
tesla_df.describe()

,Open,High,Low,Close,Adj Close,Volume
count,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000e+00
mean,22.558000,24.888000,19.130000,20.998000,20.998000,1.123578e+07
std,2.991959,3.833082,2.840906,3.332966,3.332966,6.274496e+06
min,19.000000,20.000000,15.830000,16.110001,16.110001,5.139800e+06
25%,20.000000,23.100000,17.540001,19.200001,19.200001,6.866900e+06
50%,23.000000,25.000000,18.709999,21.959999,21.959999,8.218800e+06
75%,25.000000,25.920000,20.270000,23.830000,23.830000,1.718710e+07
max,25.790001,30.420000,23.299999,23.889999,23.889999,1.876630e+07


In [5]:
tesla_df.isna().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

# Generating Features

All of the technical indicators (or features) are generated in one loop

In [6]:
#del stocks
files = os.listdir('data/raw_stocks')
stocks = {}
for file in files:
    name = file.lower().split('.')[0]
    stocks[name] = pd.read_csv('data/raw_stocks/'+file)    
    
    # Return Feature
    stocks[name]['Return'] = round(stocks[name]['Close'] / stocks[name]['Open'] - 1, 3)
    # Change Feature
    # Change of the price from previous day, absolute value
    stocks[name]['Change'] = (stocks[name].Close - stocks[name].Close.shift(1)).fillna(0)
    # Date Feature
    stocks[name]['Date'] = pd.to_datetime(stocks[name]['Date'])
    stocks[name].set_index('Date', inplace=True)
    # Volatility Feature
    stocks[name]['Volatility'] = stocks[name].Close.ewm(21).std()
    # Moving Average, 7 days
    stocks[name]['MA7'] = ma7(stocks[name])
    # Moving Average, 21 days
    stocks[name]['MA21'] = ma21(stocks[name])
    # Momentum
    stocks[name]['Momentum'] = momentum(stocks[name].Close, 3)
    # RSI (Relative Strength Index)
    stocks[name]['RSI'] = rsi(stocks[name])
    # MACD - (Moving Average Convergence/Divergence)
    stocks[name]['MACD'], stocks[name]['Signal'] = macd(stocks[name])
    # Upper Band and Lower Band for Bollinger Bands
    stocks[name]['Upper_band'], stocks[name]['Lower_band'] = bollinger_bands(stocks[name])
    stocks[name].dropna(inplace=True)
    # Saving
    stocks[name].to_csv('data/stocks/'+name+'.csv')

In [7]:
stocks['tsla'].head()

,Open,High,Low,Close,Adj Close,Volume,Return,Change,Volatility,MA7,MA21,Momentum,RSI,MACD,Signal,Upper_band,Lower_band
Date,,,,,,,,,,,,,,,,,
2010-07-28,20.549999,20.900000,20.510000,20.719999,20.719999,467200,0.008,0.170000,1.979836,20.718571,19.911904,18.290001,41.752948,-0.350607,-0.658177,24.403824,15.419985
2010-07-29,20.770000,20.879999,20.000000,20.350000,20.350000,616000,-0.020,-0.369999,1.908327,20.725714,19.743333,17.950001,40.449222,-0.337599,-0.594062,23.858103,15.628563
2010-07-30,20.200001,20.440001,19.549999,19.940001,19.940001,426900,-0.013,-0.409999,1.839567,20.685714,19.558095,17.549999,38.996148,-0.356267,-0.546503,23.226171,15.890019
2010-08-02,20.500000,20.969999,20.330000,20.920000,20.920000,718100,0.020,0.979999,1.789358,20.674286,19.508571,17.719999,44.159747,-0.288656,-0.494933,23.066887,15.950255
2010-08-03,21.000000,21.950001,20.820000,21.950001,21.950001,1230500,0.045,1.030001,1.791139,20.768572,19.639524,17.350000,49.041837,-0.150230,-0.425993,23.349330,15.929718


# Tesla News Headlines Parsing

At the moment of parsing there were 120 pages of news

'2019-01-10' - '2019-09-05'

In [ ]:
headlines_list, dates_list = [], []
for i in range(1, 120):    
    headlines, dates = get_tesla_headlines("https://www.nasdaq.com/symbol/tsla/news-headlines?page={}".format(i))
    headlines_list.append(headlines)
    dates_list.append(dates)
    time.sleep(1)

## Dataframe

In [ ]:
tesla_headlines = pd.DataFrame({'Title': [i for sub in headlines_list for i in sub], 'Date': [i for sub in dates_list for i in sub[:10]]})

## Unsupervised sentiment prediction

In [15]:
sid = SentimentIntensityAnalyzer()

In [57]:
tesla_headlines['Sentiment'] = tesla_headlines['Title'].map(lambda x: sid.polarity_scores(x)['compound'])
tesla_headlines.Date = pd.to_datetime(tesla_headlines.Date)
tesla_headlines.to_csv('data/tesla_headlines.csv')